In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import theano
from KRR_Functions import *
floatX = theano.config.floatX
import pymc3 as pm
import theano.tensor as T
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from warnings import filterwarnings
filterwarnings('ignore')
sns.set_style('white')
from sklearn import datasets

/Users/jonpvandermause/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Read in dataset.

In [2]:
# setup
SIM_NO = 201            # total number of data points
STR_PREF = '/Users/jonpvandermause/AP275/Al_Data/Store/'

pos = []
ens = []
fours = []

for n in range(SIM_NO):
    # load arrays
    en_curr = np.reshape(np.load(STR_PREF+'en_store/energy'+str(n)+'.npy'),(1))[0]
    pos_curr = np.load(STR_PREF+'pos_store/pos'+str(n)+'.npy')
    four_curr = np.load(STR_PREF+'four_store/four'+str(n)+'.npy')
    
    # store arrays
    ens.append(en_curr)
    pos_curr = pos_curr.flatten()
    pos.append(pos_curr)
    fours.append(four_curr)

# convert to np arrays
ens = np.array(ens)
pos = np.array(pos)
fours = np.abs(np.array(fours))

# Define helper functions.

In [73]:
def get_gp_model(train_X, train_Y):
    # create gaussian process model
    with pm.Model() as gp_model:

        # set hyperparameters for covariance
        rho = pm.HalfCauchy('rho', beta = 50, shape=12)
        eta = pm.HalfCauchy('eta', beta=1, shape=12)

        # define covariance function
        K = (eta**2) * pm.gp.cov.ExpQuad(12, rho) 

        # create gaussian process model -- noiseless
        lgb_gp = pm.gp.Marginal(cov_func=K)
        y_obs = lgb_gp.marginal_likelihood('y_obs', X=train_X, \
                                           y=train_Y, noise=0, is_observed=True)

    return gp_model, lgb_gp

def posterior_sample(model,size):
    # sample from the posterior
    with model:
        step=pm.Metropolis()
        lgb_gp_trace = pm.sample(size, tune=2000, step=step)
    return lgb_gp_trace

def ppc_sample(x_pred, samp_size, lgb_gp, gp):
    # get posterior predictive samples
    with gp:
        data_pred = lgb_gp.conditional('data_pred', Xnew = x_pred, pred_noise=True)
        gp_samples = pm.sample_ppc(lgb_gp_trace, vars=[data_pred], samples=samp_size)
    return data_pred, gp_samples

def get_sig(gp_samples):
    # plot linear model and posterior std
    mu = gp_samples['data_pred'].mean(0)
    sig = gp_samples['data_pred'].std(0)
    
    return mu, sig

def get_err(test_ens, mu):
    errs = test_ens - mu
    MAE = np.mean(np.abs(errs)) / 0.043 # kcal/mole
    return errs, MAE

# Training set 15.

In [71]:
M = 10
N = 201
input_dim = 12

[train_set, test_set, train_ens, test_ens, train_fours, test_fours] = get_train_test(M, N,input_dim,\
                                                                                        pos, ens, fours)

In [82]:
# move everything to Matlab -- gaussian processes are easier there
sp.io.savemat('train_set.mat',dict(train_set=train_set, test_set=test_set,\
                                  train_ens=train_ens, test_ens=test_ens, train_fours=train_fours,\
                                  test_fours=test_fours, ens=ens, pos=pos, fours=fours))


In [79]:
train_set

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.5       ,
         0.5       ,  0.5       ,  0.        ,  0.5       ,  0.5       ,
         0.5       ,  0.        ],
       [-0.0144058 ,  0.014542  , -0.0199467 ,  0.00689486,  0.470621  ,
         0.522229  ,  0.515946  ,  0.00759581,  0.483568  ,  0.491565  ,
         0.507241  ,  0.0141499 ],
       [-0.00147131, -0.00389483,  0.00791485, -0.00141156,  0.493844  ,
         0.518446  ,  0.517385  , -0.0150378 ,  0.461694  ,  0.485498  ,
         0.525089  ,  0.0119451 ],
       [-0.00832826, -0.00847987,  0.0419246 ,  0.00067264,  0.500489  ,
         0.527474  ,  0.531626  ,  0.0307514 ,  0.496386  ,  0.476029  ,
         0.477239  , -0.0657846 ],
       [ 0.038611  , -0.00145077, -0.0259325 , -0.0145651 ,  0.488923  ,
         0.497627  ,  0.47255   ,  0.00083383,  0.503712  ,  0.503405  ,
         0.511694  ,  0.0245943 ],
       [ 0.0252474 , -0.0193456 , -0.0185406 , -0.0095606 ,  0.515747  ,
         0.456669  ,  

In [72]:
[gp, lgb_gp] = get_gp_model(train_set, train_ens)

# sample from the posterior
lgb_gp_trace = posterior_sample(gp,12000)

IndexError: index 10 is out of bounds for axis 1 with size 10

In [54]:
pm.summary(lgb_gp_trace)

,mean,sd,mc_error,hpd_2.5,hpd_97.5,n_eff,Rhat
rho,3.622314e+06,3.628232e+07,384320.187747,84882.856594,8.350055e+06,7861.152306,1.000124
eta,2.229417e+02,5.069820e+01,0.875937,136.327151,3.222196e+02,3430.670954,1.000152


In [57]:
# get posterior predictive samples
[data_pred, gp_samples] = ppc_sample(test_set, 2000, lgb_gp, gp)

ValueError: incompatible dimensions

In [58]:
train_set

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.5       ,
         0.5       ,  0.5       ,  0.        ,  0.5       ,  0.5       ,
         0.5       ,  0.        ],
       [-0.0144058 ,  0.014542  , -0.0199467 ,  0.00689486,  0.470621  ,
         0.522229  ,  0.515946  ,  0.00759581,  0.483568  ,  0.491565  ,
         0.507241  ,  0.0141499 ],
       [-0.00147131, -0.00389483,  0.00791485, -0.00141156,  0.493844  ,
         0.518446  ,  0.517385  , -0.0150378 ,  0.461694  ,  0.485498  ,
         0.525089  ,  0.0119451 ],
       [-0.00832826, -0.00847987,  0.0419246 ,  0.00067264,  0.500489  ,
         0.527474  ,  0.531626  ,  0.0307514 ,  0.496386  ,  0.476029  ,
         0.477239  , -0.0657846 ],
       [ 0.038611  , -0.00145077, -0.0259325 , -0.0145651 ,  0.488923  ,
         0.497627  ,  0.47255   ,  0.00083383,  0.503712  ,  0.503405  ,
         0.511694  ,  0.0245943 ],
       [ 0.0252474 , -0.0193456 , -0.0185406 , -0.0095606 ,  0.515747  ,
         0.456669  ,  

In [47]:
test_set.shape

(191, 12)

In [49]:
train_set.shape

(10, 12)